In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv(r'../data/transformed_data.csv')
df_val = pd.read_csv(r'../data/transformed_2020_data.csv')

label_encoder = LabelEncoder()
label_encoder.fit(df['StateCode'])

X = df.drop(columns=['TotalAmountofAssistance']).values
y = df['TotalAmountofAssistance'].values
X[:, 0] = label_encoder.transform(X[:, 0])


X_val = df_val.drop(columns=['TotalAmountofAssistance']).values
y_val = df_val['TotalAmountofAssistance'].values
X_val[:, 0] = label_encoder.transform(X_val[:, 0])


X = X.astype('float32')
y = y.astype('float32')
X_val = X_val.astype('float32')
y_val = y_val.astype('float32')

In [3]:
def model_builder(hp):
  model = keras.Sequential()

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu', input_shape=[X.shape[1]]))
  model.add(keras.layers.Dense(1, activation='linear'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.metrics.RootMeanSquaredError(),
                metrics=keras.metrics.RootMeanSquaredError())

  model.summary()

  return model

In [4]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

INFO:tensorflow:Reloading Tuner from my_dir\intro_to_kt\tuner0.json


In [5]:
tuner.search(X, y, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

KeyError: <keras_tuner.engine.trial.Trial object at 0x00000182655DFEB0>

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)